In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

# folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
folder = 'data/nakdong/자동'
# folder = 'data/gum/자동'
# folder = 'data/yeongsan1/자동'

# ## 한강
# ## 자동
# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
# #자동
file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
              ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
              ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
# file_names = [['현도_2016.xlsx','현도_2017.xlsx','현도_2018.xlsx', '현도_2019.xlsx'],
#               ['대청호_2016.xlsx','대청호_2017.xlsx','대청호_2018.xlsx', '대청호_2019.xlsx'],
#               ['옥천천_2016.xlsx','옥천천_2017.xlsx','옥천천_2018.xlsx', '옥천천_2019.xlsx'],
#               ['장계_2016.xlsx','장계_2017.xlsx','장계_2018.xlsx', '장계_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['나주_2016.xlsx','나주_2017.xlsx','나주_2018.xlsx', '나주_2019.xlsx'],
#               ['지석천_2016.xlsx','지석천_2017.xlsx','지석천_2018.xlsx', '지석천_2019.xlsx'],
#               ['용봉_2016.xlsx','용봉_2017.xlsx','용봉_2018.xlsx', '용봉_2019.xlsx'],
#               ['서창교_2016.xlsx','서창교_2017.xlsx','서창교_2018.xlsx', '서창교_2019.xlsx'],
#               ['우치_2016.xlsx','우치_2017.xlsx','우치_2018.xlsx', '우치_2019.xlsx']]
              

In [7]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:', ':,28:45', ':,26:29']
nakdong = [':,2:', ':,2:12']
gum = [':,2:'] #, ':,28:45', ':,26:29']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = gum[0]
iloc_val

':,2:'

In [8]:
def make_columns(df):
    column_list = ['측정날짜', '측정소명', '수온', '수소이온농도','전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a']
    list_df = pd.DataFrame(columns=column_list)
    list_df
    df = df.drop(columns=df.columns.difference(column_list))
    new_column = list_df.columns.difference(df.columns)
#     print(new_column)
    if not new_column.empty :
        print("Make_columns")
        for i in range(new_column.shape[0]):
            df[new_column[i]] = pd.Series()
#     print('columns')
#     print(df.columns)
    return df


In [9]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [10]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
#             df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
            df_tmp = make_columns(df_tmp)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [11]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    
    return df_all, train_mean, train_std

In [12]:
def min_max_normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_min = df_all.min()
    train_max = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_min)/(train_max - train_min)       
    
    
    return df_all, train_min, train_max

In [13]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)
# df_all, train_mean, train_std = min_max_normalize(df)

도개_2016.xlsx
(8784, 12)
(8784, 12)
도개_2017.xlsx
(8760, 12)
(8760, 12)
도개_2018.xlsx
(8755, 12)
(8760, 12)
도개_2019.xlsx
(8760, 12)
(8760, 12)
신암_2016.xlsx
(8742, 11)
(8784, 11)
신암_2017.xlsx
(8744, 11)
(8760, 11)
신암_2018.xlsx
(8758, 11)
(8760, 11)
신암_2019.xlsx
(8759, 11)
(8760, 11)
회상_2016.xlsx
(8784, 27)
(8784, 27)
회상_2017.xlsx
(8742, 27)
(8760, 27)
회상_2018.xlsx
(8760, 27)
(8760, 27)
회상_2019.xlsx
(8760, 27)
(8760, 27)


In [14]:
train_mean, train_std

(수온          1.620074e+01
 수소이온농도      7.959209e+00
 전기전도도       2.361947e+02
 용존산소        1.029059e+01
 총유기탄소       2.949737e+00
 총질소         2.511964e+00
 총인          1.931998e-02
 클로로필-a      1.822085e+01
 Day sin     6.037103e-15
 Day cos     2.747682e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           9.217941
 수소이온농도       0.600598
 전기전도도       44.257904
 용존산소         2.455950
 총유기탄소        0.885916
 총질소          0.791127
 총인           0.022864
 클로로필-a      14.528678
 Day sin      0.707110
 Day cos      0.707110
 Year sin     0.707103
 Year cos     0.707117
 dtype: float64)

## 

In [15]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [16]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               out_features = None, out_num_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [17]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [18]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [19]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example2', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example3', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [20]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [21]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [22]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [23]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [24]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [25]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [28]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0     8     2     0]
 [    2     6     1     8]
 [    3     1     1    14]
 ...
 [ 6346    13     3 18669]
 [ 6349    19     4 18682]
 [ 6353     1     1 18701]]
miss_data file saved


In [29]:
dgen = GainDataGenerator(df)

pattern none


In [30]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,4.7,7.8,333.0,11.1,1.4,NaN,0.004,6.7,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.7,7.8,333.0,11.1,1.3,NaN,0.007,6.8,2.588190e-01,0.965926,-0.001950,0.999998
2,4.7,7.8,333.0,11.0,1.3,NaN,0.007,7.0,5.000000e-01,0.866025,-0.001233,0.999999
3,4.7,7.8,333.0,11.0,1.3,NaN,0.005,7.0,7.071068e-01,0.707107,-0.000516,1.000000
4,4.7,7.8,333.0,11.0,1.4,NaN,0.005,7.1,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.9,7.0,240.0,8.2,3.5,1.514,0.027,0.8,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.9,7.0,240.0,8.2,3.5,1.507,0.027,0.8,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.9,7.0,240.0,8.2,3.5,1.497,0.026,0.8,-7.071068e-01,0.707107,-0.004301,0.999991
35062,3.8,7.0,240.0,8.2,3.5,1.512,0.025,0.8,-5.000000e-01,0.866025,-0.003584,0.999994


In [31]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [32]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 12])

In [33]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  17409


In [34]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [35]:
val_performance = {}
performance = {}

In [36]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [37]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [38]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 112.6845 - disc_loss: 0.7329 - rmse: 1.0100 - val_loss: 0.9456
Epoch 2/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 89.7705 - disc_loss: 0.5245 - rmse: 0.8802 - val_loss: 0.8390
Epoch 3/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 75.1465 - disc_loss: 0.4932 - rmse: 0.8144 - val_loss: 0.8109
Epoch 4/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 65.0275 - disc_loss: 0.3782 - rmse: 0.7883 - val_loss: 0.7064
Epoch 5/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 59.4795 - disc_loss: 0.3922 - rmse: 0.7404 - val_loss: 0.7047
Epoch 6/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 47.5761 - disc_loss: 0.3899 - rmse: 0.6794 - val_loss: 0.6733
Epoch 7/2000
1/1 [==============================] - 0s 150ms/step - gen_loss: 46.8448 - disc_loss: 0.3838 - rmse: 0.6841 - val_loss: 0.6214
Ep

1/1 [==============================] - 0s 141ms/step - gen_loss: 5.3319 - disc_loss: 0.2798 - rmse: 0.2662 - val_loss: 0.3109
Epoch 60/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 4.9763 - disc_loss: 0.2833 - rmse: 0.2748 - val_loss: 0.2945
Epoch 61/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 5.3976 - disc_loss: 0.2811 - rmse: 0.2956 - val_loss: 0.2839
Epoch 62/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 4.7202 - disc_loss: 0.2754 - rmse: 0.2820 - val_loss: 0.3282
Epoch 63/2000
1/1 [==============================] - 0s 104ms/step - gen_loss: 5.8059 - disc_loss: 0.2848 - rmse: 0.2862 - val_loss: 0.3026
Epoch 64/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 4.7755 - disc_loss: 0.2792 - rmse: 0.2868 - val_loss: 0.3043
Epoch 65/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 4.8133 - disc_loss: 0.2839 - rmse: 0.2795 - val_loss: 0.3046
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 150ms/step - gen_loss: 3.5769 - disc_loss: 0.2346 - rmse: 0.2422 - val_loss: 0.2680
Epoch 118/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 4.3260 - disc_loss: 0.2352 - rmse: 0.2796 - val_loss: 0.2505
Epoch 119/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 3.4246 - disc_loss: 0.2406 - rmse: 0.2513 - val_loss: 0.2618
Epoch 120/2000
1/1 [==============================] - 0s 100ms/step - gen_loss: 3.6358 - disc_loss: 0.2414 - rmse: 0.2436 - val_loss: 0.2761
Epoch 121/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 3.8262 - disc_loss: 0.2374 - rmse: 0.2451 - val_loss: 0.2644
Epoch 122/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 3.6895 - disc_loss: 0.2393 - rmse: 0.2518 - val_loss: 0.2832
Epoch 123/2000
1/1 [==============================] - 0s 150ms/step - gen_loss: 3.5440 - disc_loss: 0.2294 - rmse: 0.2392 - val_loss: 0.2950
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 139ms/step - gen_loss: 3.6099 - disc_loss: 0.2253 - rmse: 0.2406 - val_loss: 0.2650
Epoch 176/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 3.1716 - disc_loss: 0.2292 - rmse: 0.2300 - val_loss: 0.2540
Epoch 177/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 3.2285 - disc_loss: 0.2175 - rmse: 0.2246 - val_loss: 0.2277
Epoch 178/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 3.5296 - disc_loss: 0.2126 - rmse: 0.2360 - val_loss: 0.2375
Epoch 179/2000
1/1 [==============================] - 0s 100ms/step - gen_loss: 2.9711 - disc_loss: 0.2335 - rmse: 0.2439 - val_loss: 0.2493
Epoch 180/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 3.7911 - disc_loss: 0.2268 - rmse: 0.2496 - val_loss: 0.2522
Epoch 181/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 3.1905 - disc_loss: 0.2253 - rmse: 0.2771 - val_loss: 0.2528
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 119ms/step - gen_loss: 3.0982 - disc_loss: 0.2099 - rmse: 0.2130 - val_loss: 0.2379
Epoch 234/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 2.9730 - disc_loss: 0.2144 - rmse: 0.2258 - val_loss: 0.2345
Epoch 235/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 2.8528 - disc_loss: 0.2079 - rmse: 0.2130 - val_loss: 0.2549
Epoch 236/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 3.0035 - disc_loss: 0.2125 - rmse: 0.2376 - val_loss: 0.2312
Epoch 237/2000
1/1 [==============================] - 0s 114ms/step - gen_loss: 2.6290 - disc_loss: 0.2102 - rmse: 0.2074 - val_loss: 0.2692
Epoch 238/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 2.9822 - disc_loss: 0.2051 - rmse: 0.2239 - val_loss: 0.2241
Epoch 239/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 3.0651 - disc_loss: 0.2042 - rmse: 0.2396 - val_loss: 0.2615
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 114ms/step - gen_loss: 1.9439 - disc_loss: 0.1379 - rmse: 0.2299 - val_loss: 0.2210
Epoch 1275/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 2.0161 - disc_loss: 0.1351 - rmse: 0.2587 - val_loss: 0.2083
Epoch 1276/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 1.8515 - disc_loss: 0.1282 - rmse: 0.2351 - val_loss: 0.2203
Epoch 1277/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 1.8298 - disc_loss: 0.1326 - rmse: 0.1994 - val_loss: 0.2264
Epoch 1278/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 1.7820 - disc_loss: 0.1304 - rmse: 0.2106 - val_loss: 0.2380
Epoch 1279/2000
1/1 [==============================] - 0s 94ms/step - gen_loss: 1.7907 - disc_loss: 0.1322 - rmse: 0.2134 - val_loss: 0.2206
Epoch 1280/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 1.7345 - disc_loss: 0.1344 - rmse: 0.2216 - val_loss: 0.2421
Epoch 1281/2000
1/1 [==

Epoch 1332/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 1.8921 - disc_loss: 0.1301 - rmse: 0.2416 - val_loss: 0.2400
Epoch 1333/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 1.6666 - disc_loss: 0.1287 - rmse: 0.2147 - val_loss: 0.2306
Epoch 1334/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 1.8601 - disc_loss: 0.1297 - rmse: 0.2219 - val_loss: 0.2325
Epoch 1335/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 1.6730 - disc_loss: 0.1299 - rmse: 0.1972 - val_loss: 0.2196
Epoch 1336/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 1.7164 - disc_loss: 0.1292 - rmse: 0.2086 - val_loss: 0.2207
Epoch 1337/2000
1/1 [==============================] - 0s 99ms/step - gen_loss: 1.7660 - disc_loss: 0.1340 - rmse: 0.1894 - val_loss: 0.2386
Epoch 1338/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 1.7735 - disc_loss: 0.1273 - rmse: 0.2115 - val_loss: 0.2334
Epoch 1

1/1 [==============================] - 0s 132ms/step - gen_loss: 1.8376 - disc_loss: 0.1265 - rmse: 0.2105 - val_loss: 0.2325
Epoch 1390/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 1.8751 - disc_loss: 0.1259 - rmse: 0.2266 - val_loss: 0.2375
Epoch 1391/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 1.8414 - disc_loss: 0.1234 - rmse: 0.2193 - val_loss: 0.2259
Epoch 1392/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 1.7079 - disc_loss: 0.1249 - rmse: 0.2035 - val_loss: 0.2324
Epoch 1393/2000
1/1 [==============================] - 0s 104ms/step - gen_loss: 1.7990 - disc_loss: 0.1257 - rmse: 0.2122 - val_loss: 0.2170
Epoch 1394/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 1.8802 - disc_loss: 0.1246 - rmse: 0.2262 - val_loss: 0.2300
Epoch 1395/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 1.8177 - disc_loss: 0.1259 - rmse: 0.2047 - val_loss: 0.2351
Epoch 1396/2000
1/1 [=

Epoch 1447/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 1.8802 - disc_loss: 0.1221 - rmse: 0.2252 - val_loss: 0.2315
Epoch 1448/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 1.9179 - disc_loss: 0.1263 - rmse: 0.2203 - val_loss: 0.2296
Epoch 1449/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.8748 - disc_loss: 0.1229 - rmse: 0.2280 - val_loss: 0.2300
Epoch 1450/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 1.7775 - disc_loss: 0.1250 - rmse: 0.2140 - val_loss: 0.2344
Epoch 1451/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 1.7601 - disc_loss: 0.1229 - rmse: 0.2077 - val_loss: 0.2398
Epoch 1452/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 1.8500 - disc_loss: 0.1274 - rmse: 0.2036 - val_loss: 0.2200
Epoch 1453/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 1.8334 - disc_loss: 0.1241 - rmse: 0.2032 - val_loss: 0.2298
Epoch 

1/1 [==============================] - 0s 121ms/step - gen_loss: 1.8583 - disc_loss: 0.1292 - rmse: 0.2181 - val_loss: 0.2680
Epoch 1505/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 1.9927 - disc_loss: 0.1316 - rmse: 0.2948 - val_loss: 0.2348
Epoch 1506/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 1.8147 - disc_loss: 0.1280 - rmse: 0.1950 - val_loss: 0.2571
Epoch 1507/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 1.7587 - disc_loss: 0.1271 - rmse: 0.2091 - val_loss: 0.2224
Epoch 1508/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 2.0223 - disc_loss: 0.1241 - rmse: 0.2920 - val_loss: 0.2353
Epoch 1509/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 1.8016 - disc_loss: 0.1272 - rmse: 0.2113 - val_loss: 0.2877
Epoch 1510/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 1.9151 - disc_loss: 0.1318 - rmse: 0.2139 - val_loss: 0.2039
Epoch 1511/2000
1/1 [=

**학습 loss history 출력**

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [40]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 30ms/step - loss: 0.2176


0.2175535261631012

**model save**

In [41]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [42]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
norm_df = pd.concat(df,axis=0)

In [44]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 12)
120
x.shape = (105120, 12)
x_reshape.shape = (876, 120, 12)
92854
y_true.shape= (105120, 12)
92854


(1, 120, 12)

In [45]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 2ms/step - loss: 0.0000e+00


0.0

In [46]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [47]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 12) (120, 12)
(105192, 12)


In [48]:
y_pred[~np.isnan(data)] = np.nan

In [49]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [50]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 12)
120
n= 105120
(35040, 8)
(35040, 8)


# 예측

In [51]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소     총유기탄소       총질소        총인  \
0     -1.247648 -0.265085  2.187300  0.329570 -1.749304       NaN -0.670034   
1     -1.247648 -0.265085  2.187300  0.329570 -1.862182       NaN -0.538826   
2     -1.247648 -0.265085  2.187300  0.288852 -1.862182       NaN -0.538826   
3     -1.247648 -0.265085  2.187300  0.288852 -1.862182       NaN -0.626298   
4     -1.247648 -0.265085  2.187300  0.288852 -1.749304       NaN -0.626298   
...         ...       ...       ...       ...       ...       ...       ...   
35059 -1.345284  0.234418  0.696041 -0.077605 -0.846285  0.781209 -0.582562   
35060 -1.345284  0.234418  0.650852 -0.077605 -0.733407  0.860842 -0.626298   
35061 -1.345284  0.234418  0.605662 -0.118322 -0.733407  0.851994 -0.582562   
35062 -1.366980  0.234418  0.628257 -0.118322 -0.846285  0.844410 -0.451354   
35063 -1.366980  0.234418  0.628257 -0.118322 -0.846285  0.800169 -0.626298   

         클로로필-a      

In [52]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
# print(oris[0].shape, gans[0].shape)
# print(oris[1].shape, gans[1].shape)
# print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.8)
val_no = int(total_no * 0.9)

train_slice = slice(0, train_no)
val_slice = slice(train_no, val_no)
test_slice = slice(val_no, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

In [53]:
train_df 

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-1.247648,-0.265085,2.187300,0.329570,-1.749304,-0.168463,-0.670034,-0.792973,-1.999586e-12,1.414207,...,-0.276410,-1.040758,-1.377829,0.567419,1.690214,1.306788,-0.620530,0.437396,-0.407618,-0.586485
1,-1.247648,-0.265085,2.187300,0.329570,-1.862182,-0.042474,-0.538826,-0.786090,3.660237e-01,1.366019,...,-0.451354,-1.033876,-1.377829,0.567419,1.690214,1.306788,-0.620530,0.479109,-0.495090,-0.572719
2,-1.247648,-0.265085,2.187300,0.288852,-1.862182,0.006514,-0.538826,-0.772324,7.071034e-01,1.224739,...,-0.451354,-1.033876,-1.388677,0.567419,1.712809,1.306788,-0.620530,0.500597,-0.363882,-0.586485
3,-1.247648,-0.265085,2.187300,0.288852,-1.862182,0.027945,-0.626298,-0.772324,9.999952e-01,0.999995,...,-0.101466,-1.026993,-1.377829,0.567419,1.712809,1.266071,-0.620530,0.475317,-0.451354,-0.565836
4,-1.247648,-0.265085,2.187300,0.288852,-1.749304,0.086379,-0.626298,-0.765441,1.224739e+00,0.707103,...,-0.276410,-1.013227,-1.388677,0.567419,1.712809,1.225353,-0.620530,0.494277,-0.538826,-0.565836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28027,-0.813712,0.733920,0.921990,1.266071,-0.959162,1.245104,-0.713770,1.196197,-1.366019e+00,0.366024,...,-0.670034,0.666210,-0.759469,-0.931088,-0.207752,0.411005,0.282490,-0.390536,-0.495090,-0.393762
28028,-0.824560,0.567419,0.921990,1.266071,-0.846285,1.184431,-0.582562,1.099835,-1.224739e+00,0.707103,...,-0.757506,0.645561,-0.770318,-0.931088,-0.207752,0.411005,0.282490,-0.400648,-0.451354,-0.373113
28029,-0.835408,0.567419,0.944584,1.266071,-0.959162,1.173055,-0.582562,1.106718,-9.999952e-01,0.999995,...,-0.801242,0.631796,-0.770318,-0.931088,-0.207752,0.411005,0.282490,-0.404440,-0.451354,-0.366231
28030,-0.835408,0.567419,0.921990,1.225353,-0.959162,1.243840,-0.538826,1.175548,-7.071034e-01,1.224739,...,-0.713770,0.597381,-0.770318,-0.931088,-0.207752,0.411005,0.169613,-0.425929,-0.538826,-0.359348


In [54]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [55]:
MAX_EPOCHS = 150
target_col = '용존산소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
# out_features = [target_col_idx]
out_features = [3,4,5,6,7]
out_num_features = len(out_features)
out_num_features

5

In [56]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [57]:
OUT_STEPS = 24*5

In [58]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [59]:
# train_df

In [60]:
# it = iter(wdg)

In [61]:
# x,y = next(it)
# x.shape, y.shape

In [62]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [63]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [64]:
def hour_to_day_mean(array):
    
#     print("1")
#     print(array)
    
    time = 24
    array = np.reshape(array, (array.shape[0], array.shape[1]//time, time, array.shape[2]))

    array = np.mean(array, axis=2)
    
#     print(array)

    return array

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = (input_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = (label_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = (predict_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#       predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [66]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
#     plot_out_col_index = self.column_indices[plot_out_col]
    
#     if self.label_columns:
#         label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
#     else:
#         label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
#     print(plot_col)
    
#     print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    
    sum_pred = 0
    sum_label =0
    
    o = 0
    p = 0
    
#     print(train_std[plot_col])
    
#     print(inputs.shape)
#     print(inputs[0].shape)

#     inputs = inputs[0:1]
    
# #     print(inputs)
    
#     labels = labels[0:1]
    
    predictions = model(inputs)
    
    pred = predictions.numpy()
    label = labels.numpy()
    
    for i in range(5) :
#         print(print("column : " + df_all.columns[plot_col[i]]))
        pred[:,:,i] = pred[:,:,i] * train_std[plot_col[i]] + train_mean[plot_col[i]]
        label[:,:,i] = label[:,:,i]* train_std[plot_col[i]] + train_mean[plot_col[i]]

        
#     pred_temp = hour_to_day_mean(pred[127:128,:,:]) 
    #for n in range(len(inputs)):
    #     for i in range(5) :         
     #           pred_temp = hour_to_day_mean(pred[n:n+1,:,i:i+1])
     #           label_temp = hour_to_day_mean(label[n:n+1,:,i:i+1])
    
     #for n in range(len(inputs)):           
      #      pred_temp = hour_to_day_mean(pred[n:n+1,:,:])
      #      label_temp = hour_to_day_mean(label[n:n+1,:,:])
            
#     print(pred.shape)
    
    pred = hour_to_day_mean(pred)
    label = hour_to_day_mean(label)
    
    o = np.sum(label[:,4,:],axis=0)
    
    p = np.sum(pred[:,4,:],axis=0)
    
    print("p")
    print(p)
    print("o")
    print(o)
    
       
    err = np.abs(o-p) / o *100
    
#     print(err.shape)
    
#     print(err)
    
    print(err[0])
    print(err[1])
    print(err[2])
    print(err[3])
    print(err[4])
    
    
#     print(np.sum(err,axis=0))
    
    
        
        
#     print(pred_temp)
#     np.sum(pred_temp)
#     print(pred_temp.shape)
    
#     label_temp = hour_to_day_mean(label[0][:][0])
#     print(pred[0,:,0])
#     print("adsfasdfadsfadsf")
#     print(pred_temp)
#     print(pred[0,0,0])
#     print(pred[0,0,1])
#     print(pred[0,0,2])
#     print(pred[0,0,3])
#     print(pred[0,0,4])
#       pred_temp = hour_to_day_mean(pred[0][:][0]) 
#     label_temp = hour_to_day_mean(label[0][:][0])
        
#     print(pred_temp[4])
#     print(pred_temp[4])
    
#     for n in range(len(inputs)):
#         pred_temp = hour_to_day_mean(pred[n][:][0]) 
#         label_temp = hour_to_day_mean(label[n][:][0])
        
#         print(pred_temp[4])
#         print(pred_temp[4])
    
    
#     predictions[:][:][pred_col_no] = predictions[:][:][pred_col_no] * train_std[plot_col] + train_mean[plot_col]
#     labels = labels * train_std[plot_col] + train_mean[plot_col]
    
    
#     print(predictions[0])
# #     print(plot_col)
# #     print(plot_out_col_index)
# #     pred = predictions.numpy()
# #     print(pred.shape)
# #     print(pred)
# #     print(pred[0][0][pred_col_no])

#     label = labels.numpy()
#     print(label.shape)
#     print(label)
#     print(label[0][0][pred_col_no])
    
    
    
#     print('++++++++++++++++++++++++')
#     print(np.sum(predictions.numpy()))
    
#     print(predictions.numpy().sum())
    
#     print('++++++++++++++++++++++++')
#     print(np.sum(labels.numpy()))
    
#     o = np.sum(labels.numpy())
#     p = np.sum(predictions.numpy())
#     print('++++++++++++++++++++++++')
#     print(np.abs(o-p) / o *100)

    
    
    
#     if min_max_normailze :
#         predictions = predictions * train_std[plot_col] + train_mean[plot_col]
#         labels = labels * train_std[plot_col] + train_mean[plot_col]
        
#         predictions = predictions.numpy()
#         labels = labels.numpy()
        
#         predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
#         labels = (labels - labels.min()) / (labels.max() - labels.min())
    
#     for n in range(len(inputs)):
#         pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
#         label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
#         pred_arr.append(pred_temp[4])
#         label_arr.append(label_temp[4])
        
# #         print(pred_temp[4].numpy())

        
#         sum_pred = sum_pred + pred_temp[4].numpy()
#         sum_label = sum_label + label_temp[4].numpy()
       
        
#     mae = mean_absolute_error(label_arr, pred_arr) 
#     mse = mean_squared_error(label_arr, pred_arr)
#     rmse = np.sqrt(mse)
#     mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
   

# #     print(sum_pred)
#     pbias = ( np.abs(sum_label - sum_pred) / sum_label ) * 100 
    
# #     pbias = ( (sum_label - sum_pred) / sum_label ) * 100 
    
#     print("mae:")  
#     print(mae)
    
#     print("mse:")  
#     print(mse)
    
#     print("rmse")  
#     print(rmse)
    
#     print("mape")  
#     print(mape)
    
#     print('pbias')
#     print(pbias)
    
    

WindowGenerator.compa = compa

In [67]:
train_std

수온           9.217941
수소이온농도       0.600598
전기전도도       44.257904
용존산소         2.455950
총유기탄소        0.885916
총질소          0.791127
총인           0.022864
클로로필-a      14.528678
Day sin      0.707110
Day cos      0.707110
Year sin     0.707103
Year cos     0.707117
dtype: float64

In [68]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [69]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [70]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [71]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [72]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               out_features = out_features,
                               out_num_features = out_num_features,
                               )

In [73]:
multi_val_performance = {}
multi_performance = {}

**Elman_RNN**

In [74]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [75]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [76]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 990ms/step - loss: 1.8773 - mean_absolute_error: 0.8066 - val_loss: 0.3337 - val_mean_absolute_error: 0.4615
Epoch 2/150
1/1 [==============================] - 0s 276ms/step - loss: 0.9079 - mean_absolute_error: 0.7315 - val_loss: 0.3364 - val_mean_absolute_error: 0.4557
Epoch 3/150
1/1 [==============================] - 0s 291ms/step - loss: 0.8530 - mean_absolute_error: 0.7230 - val_loss: 0.2945 - val_mean_absolute_error: 0.4302
Epoch 4/150
1/1 [==============================] - 0s 294ms/step - loss: 1.1232 - mean_absolute_error: 0.7739 - val_loss: 0.2904 - val_mean_absolute_error: 0.4243
Epoch 5/150
1/1 [==============================] - 0s 276ms/step - loss: 1.5938 - mean_absolute_error: 0.6522 - val_loss: 0.2757 - val_mean_absolute_error: 0.4110
Epoch 6/150
1/1 [==============================] - 0s 273ms/step - loss: 1.2882 - mean_absolute_error: 0.6858 - val_loss: 0.3054 - val_mean_absolute_error: 0.4305
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 266ms/step - loss: 0.7086 - mean_absolute_error: 0.3560 - val_loss: 0.2278 - val_mean_absolute_error: 0.3519
Epoch 52/150
1/1 [==============================] - 0s 301ms/step - loss: 0.3083 - mean_absolute_error: 0.3597 - val_loss: 0.2166 - val_mean_absolute_error: 0.3463
Epoch 53/150
1/1 [==============================] - 0s 290ms/step - loss: 0.3243 - mean_absolute_error: 0.3705 - val_loss: 0.2218 - val_mean_absolute_error: 0.3544
Epoch 54/150
1/1 [==============================] - 0s 271ms/step - loss: 0.2608 - mean_absolute_error: 0.3497 - val_loss: 0.2042 - val_mean_absolute_error: 0.3399
Epoch 55/150
1/1 [==============================] - 0s 275ms/step - loss: 0.2743 - mean_absolute_error: 0.3470 - val_loss: 0.2269 - val_mean_absolute_error: 0.3500
Epoch 56/150
1/1 [==============================] - 0s 285ms/step - loss: 0.3469 - mean_absolute_error: 0.3878 - val_loss: 0.2208 - val_mean_absolute_error: 0.3397
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 250ms/step - loss: 0.6710 - mean_absolute_error: 0.3343 - val_loss: 0.2278 - val_mean_absolute_error: 0.3606
Epoch 102/150
1/1 [==============================] - 0s 288ms/step - loss: 0.6898 - mean_absolute_error: 0.3318 - val_loss: 0.2042 - val_mean_absolute_error: 0.3363
Epoch 103/150
1/1 [==============================] - 0s 287ms/step - loss: 0.2598 - mean_absolute_error: 0.3416 - val_loss: 0.2247 - val_mean_absolute_error: 0.3443
Epoch 104/150
1/1 [==============================] - 0s 293ms/step - loss: 0.2651 - mean_absolute_error: 0.3218 - val_loss: 0.2116 - val_mean_absolute_error: 0.3494
Epoch 105/150
1/1 [==============================] - 0s 284ms/step - loss: 0.6693 - mean_absolute_error: 0.3308 - val_loss: 0.1926 - val_mean_absolute_error: 0.3249
Epoch 106/150
1/1 [==============================] - 0s 306ms/step - loss: 0.2452 - mean_absolute_error: 0.3175 - val_loss: 0.2302 - val_mean_absolute_error: 0.3533
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 92ms/step - loss: 0.4885 - mean_absolute_error: 0.5331
val performance = [0.2508184313774109, 0.37921351194381714]
test performance =  [0.4885083734989166, 0.5331301093101501]


In [77]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [79]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [80]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 935ms/step - loss: 0.9750 - mean_absolute_error: 0.7501 - val_loss: 0.4087 - val_mean_absolute_error: 0.5104
Epoch 2/150
1/1 [==============================] - 0s 304ms/step - loss: 1.5088 - mean_absolute_error: 0.8171 - val_loss: 0.3828 - val_mean_absolute_error: 0.4865
Epoch 3/150
1/1 [==============================] - 0s 319ms/step - loss: 1.3282 - mean_absolute_error: 0.7285 - val_loss: 0.3114 - val_mean_absolute_error: 0.4470
Epoch 4/150
1/1 [==============================] - 0s 345ms/step - loss: 1.8571 - mean_absolute_error: 0.7617 - val_loss: 0.3436 - val_mean_absolute_error: 0.4653
Epoch 5/150
1/1 [==============================] - 0s 327ms/step - loss: 0.8561 - mean_absolute_error: 0.7032 - val_loss: 0.3430 - val_mean_absolute_error: 0.4597
Epoch 6/150
1/1 [==============================] - 0s 306ms/step - loss: 1.1696 - mean_absolute_error: 0.6616 - val_loss: 0.2965 - val_mean_absolute_error: 0.4270
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 321ms/step - loss: 0.4074 - mean_absolute_error: 0.4139 - val_loss: 0.2302 - val_mean_absolute_error: 0.3612
Epoch 52/150
1/1 [==============================] - 0s 305ms/step - loss: 0.3509 - mean_absolute_error: 0.4166 - val_loss: 0.2535 - val_mean_absolute_error: 0.3614
Epoch 53/150
1/1 [==============================] - 0s 343ms/step - loss: 0.2924 - mean_absolute_error: 0.3828 - val_loss: 0.2306 - val_mean_absolute_error: 0.3475
Epoch 54/150
1/1 [==============================] - 0s 322ms/step - loss: 0.3922 - mean_absolute_error: 0.3996 - val_loss: 0.2265 - val_mean_absolute_error: 0.3594
Epoch 55/150
1/1 [==============================] - 0s 291ms/step - loss: 0.7876 - mean_absolute_error: 0.4021 - val_loss: 0.2256 - val_mean_absolute_error: 0.3494
Epoch 56/150
1/1 [==============================] - 0s 306ms/step - loss: 0.7918 - mean_absolute_error: 0.4143 - val_loss: 0.2174 - val_mean_absolute_error: 0.3453
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 318ms/step - loss: 0.7504 - mean_absolute_error: 0.3630 - val_loss: 0.2263 - val_mean_absolute_error: 0.3538
Epoch 102/150
1/1 [==============================] - 0s 291ms/step - loss: 0.2507 - mean_absolute_error: 0.3238 - val_loss: 0.2095 - val_mean_absolute_error: 0.3312
Epoch 103/150
1/1 [==============================] - 0s 324ms/step - loss: 0.2438 - mean_absolute_error: 0.3237 - val_loss: 0.2432 - val_mean_absolute_error: 0.3442
Epoch 104/150
1/1 [==============================] - 0s 298ms/step - loss: 0.3417 - mean_absolute_error: 0.3474 - val_loss: 0.1972 - val_mean_absolute_error: 0.3246
Epoch 105/150
1/1 [==============================] - 0s 302ms/step - loss: 0.6791 - mean_absolute_error: 0.3333 - val_loss: 0.2282 - val_mean_absolute_error: 0.3564
Epoch 106/150
1/1 [==============================] - 0s 306ms/step - loss: 0.3443 - mean_absolute_error: 0.3493 - val_loss: 0.2012 - val_mean_absolute_error: 0.3268
Epoch 107/150
1/1 [=====

100/100 [==============================] - 10s 101ms/step - loss: 0.5193 - mean_absolute_error: 0.5514
val performance = [0.22716306149959564, 0.34792500734329224]
test performance =  [0.5193037986755371, 0.5514184832572937]


**LSTM**

In [81]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [82]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 3s 3s/step - loss: 1.0664 - mean_absolute_error: 0.7900 - val_loss: 0.3800 - val_mean_absolute_error: 0.4876
Epoch 2/150
1/1 [==============================] - 0s 255ms/step - loss: 0.9862 - mean_absolute_error: 0.7684 - val_loss: 0.3536 - val_mean_absolute_error: 0.4807
Epoch 3/150
1/1 [==============================] - 0s 259ms/step - loss: 1.8432 - mean_absolute_error: 0.7700 - val_loss: 0.3875 - val_mean_absolute_error: 0.4947
Epoch 4/150
1/1 [==============================] - 0s 267ms/step - loss: 1.4524 - mean_absolute_error: 0.7822 - val_loss: 0.3462 - val_mean_absolute_error: 0.4688
Epoch 5/150
1/1 [==============================] - 0s 240ms/step - loss: 0.8378 - mean_absolute_error: 0.7122 - val_loss: 0.3340 - val_mean_absolute_error: 0.4577
Epoch 6/150
1/1 [==============================] - 0s 248ms/step - loss: 1.7767 - mean_absolute_error: 0.7437 - val_loss: 0.3464 - val_mean_absolute_error: 0.4657
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 285ms/step - loss: 0.8069 - mean_absolute_error: 0.4119 - val_loss: 0.2940 - val_mean_absolute_error: 0.3882
Epoch 52/150
1/1 [==============================] - 0s 253ms/step - loss: 0.3323 - mean_absolute_error: 0.3878 - val_loss: 0.2767 - val_mean_absolute_error: 0.3934
Epoch 53/150
1/1 [==============================] - 0s 289ms/step - loss: 0.4458 - mean_absolute_error: 0.4260 - val_loss: 0.3374 - val_mean_absolute_error: 0.4313
Epoch 54/150
1/1 [==============================] - 0s 271ms/step - loss: 0.7657 - mean_absolute_error: 0.3882 - val_loss: 0.2602 - val_mean_absolute_error: 0.3761
Epoch 55/150
1/1 [==============================] - 0s 261ms/step - loss: 0.3155 - mean_absolute_error: 0.3737 - val_loss: 0.2845 - val_mean_absolute_error: 0.3934
Epoch 56/150
1/1 [==============================] - 0s 238ms/step - loss: 1.6106 - mean_absolute_error: 0.4069 - val_loss: 0.3191 - val_mean_absolute_error: 0.4150
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 258ms/step - loss: 0.2850 - mean_absolute_error: 0.3274 - val_loss: 0.3460 - val_mean_absolute_error: 0.4226
Epoch 102/150
1/1 [==============================] - 0s 228ms/step - loss: 0.6716 - mean_absolute_error: 0.3246 - val_loss: 0.3521 - val_mean_absolute_error: 0.4309
Epoch 103/150
1/1 [==============================] - 0s 249ms/step - loss: 0.1940 - mean_absolute_error: 0.3012 - val_loss: 0.3315 - val_mean_absolute_error: 0.4133
Epoch 104/150
1/1 [==============================] - 0s 268ms/step - loss: 0.2029 - mean_absolute_error: 0.3058 - val_loss: 0.4320 - val_mean_absolute_error: 0.4642
Epoch 105/150
1/1 [==============================] - 0s 270ms/step - loss: 0.2233 - mean_absolute_error: 0.3057 - val_loss: 0.4044 - val_mean_absolute_error: 0.4607
Epoch 106/150
1/1 [==============================] - 0s 239ms/step - loss: 0.6303 - mean_absolute_error: 0.3247 - val_loss: 0.3426 - val_mean_absolute_error: 0.4086
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 124ms/step - loss: 0.5516 - mean_absolute_error: 0.5160
val performance = [0.5516383051872253, 0.5160062909126282]
test performance =  [0.6464539766311646, 0.608633816242218]


**CNN model**

In [83]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [84]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 1s 758ms/step - loss: 1.1477 - mean_absolute_error: 0.8197 - val_loss: 0.3761 - val_mean_absolute_error: 0.4859
Epoch 2/150
1/1 [==============================] - 0s 226ms/step - loss: 0.9776 - mean_absolute_error: 0.7616 - val_loss: 0.3653 - val_mean_absolute_error: 0.4799
Epoch 3/150
1/1 [==============================] - 0s 253ms/step - loss: 0.7180 - mean_absolute_error: 0.6506 - val_loss: 0.3059 - val_mean_absolute_error: 0.4321
Epoch 4/150
1/1 [==============================] - 0s 238ms/step - loss: 1.1411 - mean_absolute_error: 0.6245 - val_loss: 0.3560 - val_mean_absolute_error: 0.4626
Epoch 5/150
1/1 [==============================] - 0s 271ms/step - loss: 0.4912 - mean_absolute_error: 0.5224 - val_loss: 0.2746 - val_mean_absolute_error: 0.4036
Epoch 6/150
1/1 [==============================] - 0s 228ms/step - loss: 0.4106 - mean_absolute_error: 0.4778 - val_loss: 0.3442 - val_mean_absolute_error: 0.4515
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 229ms/step - loss: 0.8013 - mean_absolute_error: 0.3624 - val_loss: 0.2142 - val_mean_absolute_error: 0.3243
Epoch 52/150
1/1 [==============================] - 0s 252ms/step - loss: 0.3251 - mean_absolute_error: 0.3600 - val_loss: 0.2423 - val_mean_absolute_error: 0.3437
Epoch 53/150
1/1 [==============================] - 0s 231ms/step - loss: 0.2455 - mean_absolute_error: 0.3174 - val_loss: 0.1986 - val_mean_absolute_error: 0.3169
Epoch 54/150
1/1 [==============================] - 0s 256ms/step - loss: 0.1767 - mean_absolute_error: 0.2920 - val_loss: 0.2127 - val_mean_absolute_error: 0.3287
Epoch 55/150
1/1 [==============================] - 0s 249ms/step - loss: 0.3169 - mean_absolute_error: 0.3491 - val_loss: 0.1796 - val_mean_absolute_error: 0.3004
Epoch 56/150
1/1 [==============================] - 0s 221ms/step - loss: 0.2681 - mean_absolute_error: 0.3284 - val_loss: 0.1889 - val_mean_absolute_error: 0.3039
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 259ms/step - loss: 0.3111 - mean_absolute_error: 0.3395 - val_loss: 0.2303 - val_mean_absolute_error: 0.3443
Epoch 102/150
1/1 [==============================] - 0s 238ms/step - loss: 0.2171 - mean_absolute_error: 0.3039 - val_loss: 0.2896 - val_mean_absolute_error: 0.3827
Epoch 103/150
1/1 [==============================] - 0s 261ms/step - loss: 0.7186 - mean_absolute_error: 0.3305 - val_loss: 0.2541 - val_mean_absolute_error: 0.3576
Epoch 104/150
1/1 [==============================] - 0s 236ms/step - loss: 0.2244 - mean_absolute_error: 0.2998 - val_loss: 0.2537 - val_mean_absolute_error: 0.3514
Epoch 105/150
1/1 [==============================] - 0s 219ms/step - loss: 0.1739 - mean_absolute_error: 0.2832 - val_loss: 0.2379 - val_mean_absolute_error: 0.3434
Epoch 106/150
1/1 [==============================] - 0s 218ms/step - loss: 0.1993 - mean_absolute_error: 0.2829 - val_loss: 0.2029 - val_mean_absolute_error: 0.3173
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 125ms/step - loss: 0.2461 - mean_absolute_error: 0.3394
val performance = [0.24611487984657288, 0.3393770456314087]
test performance =  [0.6612145304679871, 0.6017182469367981]


In [85]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 109ms/step - loss: 0.3198 - mean_absolute_error: 0.3848
val performance = [0.31981220841407776, 0.38475677371025085]
test performance =  [0.6063066124916077, 0.5782169699668884]


**performance**

In [86]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
multi_window.compa(elman_model, plot_col=out_features, windows=multi_window.example3)

p
[1321.0266    415.8347    405.36856     4.429245 2919.1736  ]
o
[1250.7184     400.73453    387.8211       3.6067436 3270.115    ]
5.6214275
3.7681203
4.5246263
22.804543
10.731776


In [98]:
train_mean

수온          1.620074e+01
수소이온농도      7.959209e+00
전기전도도       2.361947e+02
용존산소        1.029059e+01
총유기탄소       2.949737e+00
총질소         2.511964e+00
총인          1.931998e-02
클로로필-a      1.822085e+01
Day sin     6.037103e-15
Day cos     2.747682e-15
Year sin   -5.681266e-08
Year cos    2.053380e-05
dtype: float64

In [101]:
25.982983 / 0.001931998

13448.762886918103